In [38]:
import pandas as pd
import numpy as np
import os

In [48]:
#ディレクトリ直下のファイルのパスを取得する
data_name = []
firm_data=pd.DataFrame()
for i in os.listdir():
    if i.endswith('.xlsx') or i.endswith('.xls'):
        # data_name.append(i)
        # yearを付与してデータを結合する
        data_i=pd.read_excel(i)
        data_i['year']=i[2:6]
        if i[2:6]=="2023":
            continue
        firm_data = pd.concat([firm_data,data_i],axis=0,ignore_index=True)

In [49]:
firm_data=firm_data[firm_data["メーカー名"].notna()]
firm_data.shape

(56725, 21)

大正薬品工業は名前を変えた

In [50]:
firm_data["year"]

1        2014
2        2014
3        2014
4        2014
5        2014
         ... 
62197    2022
62198    2022
62199    2022
62200    2022
62201    2022
Name: year, Length: 56725, dtype: object

In [51]:
dup=firm_data[firm_data.duplicated(["薬価基準収載医薬品コード","year"])]
#同じ年で名前が変わっている? 2023まで入れればね
for id in set(dup["薬価基準収載医薬品コード"]):
    df_id=dup[dup["薬価基準収載医薬品コード"]==id]
    year=set(df_id["year"])
    maker=set(df_id["メーカー名"])
    if len(maker)>1:
        print(id,year)
        print(maker)

In [52]:
len(set(firm_data["メーカー名"])),len(set(firm_data["薬価基準収載医薬品コード"])),sum(firm_data["メーカー名"].isna())

(351, 17514, 0)

In [53]:
print(set(firm_data["メーカー名"]))

{'東亞合成', 'キョーリンリメディオ', '高田製薬', '協和新薬', '味の素製薬', '山崎帝國堂', 'ツキオカフィルム製薬', 'コーアバイオテックベイ', '山善製薬', '大蔵製薬', '栃本天海堂', 'ジョンソン・エンド・ジョンソン', 'ヤクルト本社', '小野薬品工業', '大和薬品工業', '金田直隆商店', 'ゼリア新薬工業', 'ＤＳファーマバイオメディカル', '東洋製薬化成', 'アボットジャパン', '大峰堂薬品工業', '司生堂製薬', '東洋カプセル', '日本ケミファ', '大塚製薬工場', '東レ・メディカル', '武田テバファーマ', '化研生薬', 'フェリング・ファーマ', '資生堂', '摩耶堂製薬', '武田テバ薬品', '伏見製薬所', 'ＤＳファーマプロモ', '安西メディカル', '遼東化学工業', '化学及血清療法研究所', '久光製薬', 'ノーベルファーマ', 'サンファーマ', '生晃栄養薬品', 'ムネ製薬', 'フヂミ製薬所', 'オルガノン', 'Ｍｅｉｊｉ\u3000Ｓｅｉｋａ\u3000ファルマ', '三全製薬', '辰巳化学', '佐藤製薬', '日医工', '第一薬品産業', '兼一薬品工業', 'アサヒフードアンドヘルスケア', '三友薬品', '富士フイルムファーマ', '京都薬品工業', 'エア・ウォーター', 'サンド', '日本イーライリリー', '杏林製薬', 'リョートーファイン', 'シャイアー・ジャパン', '日医工ファーマ', 'ノボ\u3000ノルディスク\u3000ファーマ', 'メディサ新薬', '富士化学工業', 'シオノギファーマ', '東菱薬品工業', '日興製薬', '松浦薬業', 'ＭＳＤ', '大原薬品工業', 'トーヨーケム', 'ニッテン', '日本家庭用塩', '三和化学研究所', 'アムジェン', '佐藤薬品工業', 'メドレックス', 'グラクソ・スミスクライン・コンシューマー・ヘルスケア・ジャパン', '大興製薬', '雪印メグミルク', '富士薬品', '参天アイケア', '持田製薬販売', 'アステラス製薬', 'ヤンセンファーマ', 'サンノーバ', '東和製薬：和歌山', '共和クリティケア', '康和薬通', '富士フイルム

In [54]:
data_n=pd.read_csv("../merged/len3_ndb_blp_DN.csv",encoding="shift-jis",index_col=0)
# data_n=data_n[data_n["year"]>=2018]
print("initial",len(set(data_n["薬価基準収載医薬品コード"])))

initial 2671


In [55]:
data_n.columns

Index(['薬効分類', '薬効分類名称', '医薬品コード', '医薬品名', '薬価基準収載医薬品コード', '薬価', '後発品区分', '総計',
       'year', 'in_hospital',
       ...
       'id_625.0', 'id_629.0', 'id_799.0', 'q_share', 'r_share',
       'generic_share_q', 'generic_share_r', 'lag_generic_share_q',
       'mean_price_g', 'mean_price_b'],
      dtype='object', length=112)

In [56]:
len(set(data_n["薬価基準収載医薬品コード"]) - set(firm_data["薬価基準収載医薬品コード"]))

290

In [57]:
data_n["医薬品コード"]

0        620007096.0
1        620007095.0
2        620007096.0
3        620007095.0
4        661140081.0
            ...     
25793    662710001.0
25794    620003492.0
25795    620003491.0
25796    620003490.0
25797    662710001.0
Name: 医薬品コード, Length: 25798, dtype: float64